# Load data

In [102]:
import scipy.io
saved  = scipy.io.loadmat("../matlab/utility.mat")
attacker_utility = saved["attacker_utility"]
defender_utility = saved["defender_utility"]

# Configure numpy display options

In [103]:
import numpy as np
import scipy.optimize

np.set_printoptions(
    suppress=True,
    linewidth=180
)

num_options = len(attacker_utility)

# Setup solver for defender

In [104]:
c = np.zeros(num_options+1)
c[0] = -1
print(c)

[-1.  0.  0. ...  0.  0.  0.]


In [105]:
A_ub = np.ones((num_options, num_options+1))
A_ub[:,1:] = defender_utility*-1
print(A_ub)

[[  1.       3.5424   8.64   ... 999.     999.     999.    ]
 [  1.      11.68     1.168  ... 999.     999.     999.    ]
 [  1.       5.49     5.49   ... 999.     999.     999.    ]
 ...
 [  1.      35.2524  40.35   ... 999.     999.     999.    ]
 [  1.      43.39    32.878  ... 999.     999.     999.    ]
 [  1.      46.9324  41.518  ... 999.     999.     999.    ]]


In [106]:
print(max(A_ub.flatten()))
print(min(A_ub.flatten()))

999.0
0.05759999999999997


In [107]:
# b = np.zeros((num_options,1))
b_ub = np.zeros(num_options)
print(b_ub)

[0. 0. 0. ... 0. 0. 0.]


In [108]:
A_eq = np.ones((1, num_options+1))
A_eq[0][0] = 0
print(A_eq)

[[0. 1. 1. ... 1. 1. 1.]]


In [109]:
b_eq = 1
print(b_eq)

1


In [110]:
lb = np.zeros(num_options+1)
lb[0] = -10000

ub = np.ones(num_options+1)
ub[0] = 10000

bounds = np.asarray([lb, ub]).transpose()
print(bounds)

[[-10000.  10000.]
 [     0.      1.]
 [     0.      1.]
 ...
 [     0.      1.]
 [     0.      1.]
 [     0.      1.]]


# Solve for defender

In [111]:
# zz = []
result = scipy.optimize.linprog(
    c=c,
    A_ub=A_ub,
    b_ub=b_ub,
    A_eq=A_eq,
    b_eq=b_eq,
    bounds=bounds,
    method="highs",
    # method="highs",
    # options={"presolve":False},
    # callback = lambda x: zz.append(x.x)
)
if result.success:
    print(result.x)
else:
    print(result)
# todo: trim result to not include bad states? idk
# todo: don't evaluate for bad states, cut from matrix I guess
# this has -999 for first item because there are no states where defender doesn't have -999 in a row since the attacker can't make certain attacks

[-21.2488   0.       0.     ...   0.       0.       0.    ]


# Evaluate defender results

In [112]:
example = np.asarray([
    [1,2,3,4],
    [5,6,7,8],
    [9,10,11,12],
    [13,14,15,16],
])

In [113]:
soln = np.asarray([1,2,3,4])
print(example * soln)
# mul = np.broadcast_to(soln, (len(soln),) + soln.shape)
# print(mul, mul.sum(axis=0))

[[ 1  4  9 16]
 [ 5 12 21 32]
 [ 9 20 33 48]
 [13 28 45 64]]


In [126]:
temp = np.asarray([[1,9,3],[4,5,6]])
print(temp.max(axis=0))
print(temp.max())

[4 9 6]
9


In [127]:
soln = result.x[1:]
temp = (defender_utility * soln)
print("min", temp.min(), "max", temp.max())


# # multiply each option by the probability that the other person takes the option
# # then find the best one of those
# actual = (defender_utility * soln).sum(axis=0)
# #  @ result.x[1:]
# print(actual, min(actual))

# import scipy.io
# scipy.io.savemat("../matlab/temp.mat", {
#     "temp": actual,
# })

min -21.2488 max -0.0


In [48]:
guess = defender_utility @ [0,1,0,0,0,0,0,0,0,0,0,0,0,0,0]
print(guess, sum(guess))

[ -7.11    -0.3936  -8.      -5.4     -7.5036 -15.11   -12.51    -8.3936  -5.7936 -13.4    -15.5036 -12.9036 -20.51   -13.7936 -20.9036] -167.2288


# Setup solver for attacker

In [49]:
A_ub = np.ones((num_options, num_options+1))
A_ub[:,1:] = attacker_utility.transpose()*-1
print(A_ub)

[[  1.      -2.2041 -13.12    -8.      -5.4    -15.3241 -10.2041  -7.6041 -21.12   -18.52   -13.4    -23.3241 -20.7241 999.     999.     999.    ]
 [  1.      -7.11    -0.3936  -8.      -5.4     -7.5036 -15.11   -12.51    -8.3936  -5.7936 -13.4    -15.5036 -12.9036 999.     999.     999.    ]
 [  1.      -7.11   -13.12    -1.84    -5.4    -20.23    -8.95   -12.51   -14.96   -18.52    -7.24   -22.07   -25.63   999.     999.     999.    ]
 [  1.      -7.11   -13.12    -8.      -1.242  -20.23   -15.11    -8.352  -21.12   -14.362   -9.242  -28.23   -21.472  999.     999.     999.    ]
 [  1.      -2.2041  -0.3936  -8.      -5.4     -2.5977 -10.2041  -7.6041  -8.3936  -5.7936 -13.4    -10.5977  -7.9977 999.     999.     999.    ]
 [  1.      -2.2041 -13.12    -1.84    -5.4    -15.3241  -4.0441  -7.6041 -14.96   -18.52    -7.24   -17.1641 -20.7241 999.     999.     999.    ]
 [  1.      -2.2041 -13.12    -8.      -1.242  -15.3241 -10.2041  -3.4461 -21.12   -14.362   -9.242  -23.3241 -16.5661

# Solve for attacker

In [50]:
# zz = []
result = scipy.optimize.linprog(
    c=c,
    A_ub=A_ub,
    b_ub=b_ub,
    A_eq=A_eq,
    b_eq=b_eq,
    bounds=bounds,
    method="simplex",
    # method="highs",
    # options={"presolve":False},
    # callback = lambda x: zz.append(x.x)
)
if result.success:
    print(result.x)
else:
    print(result)
# todo: trim result to not include bad states? idk
# todo: don't evaluate for bad states, cut from matrix I guess
# this has -999 for first item because there are no states where defender doesn't have -999 in a row since the attacker can't make certain attacks

[4.4377 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     1.     0.     0.     0.     0.    ]


# Evaluate attacker results

In [129]:
# actual = defender_utility @ result.x[1:]
# todo: investigate
# actual = (defender_utility * np.broadcast_to(soln, (len(soln),) + soln.shape).transpose()).sum(axis=0)
# print(actual, max(actual))

In [130]:
guess = attacker_utility.transpose() @ [0,1,0,0,0,0,0,0,0,0,0,0,0,0,0]
print(guess, sum(guess))

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 15 is different from 1023)